In [1]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import pyreax

except ModuleNotFoundError:
    # relative import; better to pip install subctrl
    import sys
    sys.path.append("..")
    import pyreax

/u/nlp/anaconda/main/anaconda3/envs/wuzhengx-310/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
import json
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
import torch, pyreft
from pyvene import (
    IntervenableModel,
    ConstantSourceIntervention,
    SourcelessIntervention,
    TrainableIntervention,
    DistributedRepresentationIntervention,
)

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import get_scheduler

from circuitsvis.tokens import colored_tokens
from IPython.core.display import display, HTML
from pyreax import (
    EXAMPLE_TAG, 
    ReAXFactory, 
    MaxReLUIntervention, 
    SubspaceAdditionIntervention, 
    JumpReLUSAECollectIntervention,
    make_data_module, 
    save_reax,
    load_reax,
    load_sae,
)
from pyreax import (
    set_decoder_norm_to_unit_norm, 
    remove_gradient_parallel_to_decoder_directions,
    gather_residual_activations,
    get_lr
)

/tmp/wuzhengx/ipykernel_111067/408558312.py:18: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# Load lm.
model_name = "google/gemma-2-2b"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cpu")
model.config.use_cache = False
model = model.cuda()

tokenizer =  AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
training_df, concept_metadata, weights = load_reax("./tmp_save/")
sae_weights = load_sae(concept_metadata)

In [5]:
LAYER = 20

reax_intervention = MaxReLUIntervention(
    embed_dim=model.config.hidden_size, low_rank_dimension=weights.shape[0],
)
reax_intervention.proj.weight.data = weights.data
_ = reax_intervention.cuda()
pv_reax_model = IntervenableModel({
   "component": f"model.layers[{LAYER}].output",
   "intervention": reax_intervention}, model=model)

sae_intervention = JumpReLUSAECollectIntervention(
    embed_dim=sae_weights['W_enc'].shape[0],
    low_rank_dimension=sae_weights['W_enc'].shape[1]
)
sae_intervention.load_state_dict(sae_weights, strict=False)
_ = sae_intervention.cuda()
pv_sae_model = IntervenableModel({
   "component": f"model.layers[{LAYER}].output",
   "intervention": sae_intervention}, model=model)

In [35]:
dump_dir = "./tmp"
val_n = 10
n_decimal = 3
reax_topk = 10

all_validation_dfs = []

for meta in concept_metadata:
    
    meta_dict = json.loads(meta)
    concept = meta_dict["concept"]
    print("Testing with concept:", concept)
    
    reax_id = int(meta_dict["_id"])
    sae_id = int(meta_dict["sae_concept"].split("/")[-1])

    # test prompt
    reax_factory = ReAXFactory(
        model, tokenizer,
        concepts=[concept], 
        dump_dir=dump_dir)

    positive_df = reax_factory.create_eval_df(n=val_n, category="positive")
    negative_df = reax_factory.create_eval_df(n=val_n, category="negative")
    hard_negative_df = reax_factory.create_eval_df(n=val_n, category="hard negative")
    
    validation_df = pd.concat([positive_df, negative_df, hard_negative_df], axis=0)
    all_sae_acts = []
    all_reax_acts = []
    all_sae_max_act = []
    all_reax_max_act = []
    for _, row in validation_df.iterrows():
        inputs = tokenizer.encode(
            row["input"], return_tensors="pt", add_special_tokens=True).to("cuda")
        # sae acts
        sae_acts = pv_sae_model.forward(
            {"input_ids": inputs}, return_dict=True
        ).collected_activations[0][1:, sae_id].data.cpu().numpy().tolist() # no bos token
        sae_acts = [round(x, n_decimal) for x in sae_acts]
        max_sae_act = max(sae_acts)
        
        # reax acts
        reax_in = gather_residual_activations(model, LAYER, inputs)
        reax_acts, _ = reax_intervention.encode(
            reax_in[:,1:], # no bos token
            subspaces={
                "input_subspaces": torch.tensor([reax_id])}, k=reax_topk)
        reax_acts = reax_acts.flatten().data.cpu().numpy().tolist()
        reax_acts = [round(x, n_decimal) for x in reax_acts]
        max_reax_act = max(reax_acts)
        
        all_sae_acts += [sae_acts]
        all_reax_acts += [reax_acts]
        all_sae_max_act += [max_sae_act]
        all_reax_max_act += [max_reax_act]
        
    validation_df['sae_acts'] = all_sae_acts
    validation_df['reax_acts'] = all_reax_acts
    validation_df['max_sae_act'] = all_sae_max_act
    validation_df['max_reax_act'] = all_reax_max_act
    validation_df['reax_id'] = reax_id
    validation_df['sae_id'] = sae_id
    all_validation_dfs += [validation_df]

all_validation_df = pd.concat(all_validation_dfs, axis=0)

Less than 2 concepts are provided. Only eval mode is allowed.
Prepare contrast concepts.


Testing with concept: terms related to artificiality and deception


Fectching 2 contrast concepts for concept: terms related to artificiality and deception
Creating dataframe.
Creating dataframe.
Creating dataframe.
Less than 2 concepts are provided. Only eval mode is allowed.
Prepare contrast concepts.


Testing with concept: terms related to employment and employees


Fectching 0 contrast concepts for concept: terms related to employment and employees
Creating dataframe.
Creating dataframe.
Creating dataframe.


In [37]:
all_validation_df.to_csv("./tmp/all_validation_df.csv")

In [40]:
# Generate the HTML content with the dropdown for selecting REAX ID
html_content_interactive = generate_html_with_concept(
    pd.read_csv("./tmp/all_validation_df.csv"), 
    tokenizer
)

# Save the updated HTML content to a new file
output_file_interactive = './tmp/highlighted_text.html'
with open(output_file_interactive, 'w') as file:
    file.write(html_content_interactive)